<a href="https://colab.research.google.com/github/bilalProgTech/mtech-nmims/blob/master/speech-recognition/Lab-Work/20221019-Lab-11-MTech-AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'
!kaggle competitions download -c tensorflow-speech-recognition-challenge
!unzip *.zip
!7za x 'train.7z'

In [2]:
import tensorflow as tf
import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix
import IPython.display as ipd

In [ ]:
files = []
target_series = []
for dirname, _, filenames in os.walk('/content/train/audio/'):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        target = filepath.split('/')[-2]
        if target == 'right' or target == 'seven' or target == 'three':
            target_series.append(target)
            files.append(filepath)
data = pd.DataFrame(target_series, columns=['target'])
data['filename'] = files
data = data.sample(frac=1)
data = data.reset_index(drop=True)
data.head()

In [ ]:
path = '/content/train/audio/three/1b4c9b89_nohash_1.wav'
x , sr = librosa.load(path)
plt.figure(figsize=(20, 5))
plt.title('Waveplot')
librosa.display.waveplot(x, sr=sr)
plt.show()
ipd.Audio(path)

In [31]:
train, test = train_test_split(data, test_size=0.20)

In [124]:
file_count = 0
x_train_new = np.zeros((train.shape[0], 256, 20), dtype=np.float64)
for count_, path in zip(range(train.shape[0]), train['filename'].values):
    x , sr = librosa.load(path)
    try:
        mfcc = librosa.feature.mfcc(y=x, sr=sr, n_fft=256, n_mfcc=20)
        x_train_new[count_, :, :] = mfcc.T
    except:
        continue
        print('Error reading audio')
    count_ += 1

In [ ]:
x_train_new.shape

In [126]:
tf.keras.backend.clear_session()
class SpeechAE(tf.keras.models.Model):
  def __init__(self):
    super(SpeechAE, self).__init__()
    self.encoder = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(128, 20, 1)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(2, 2)])

    self.decoder = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.UpSampling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.UpSampling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = SpeechAE()

In [127]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
history = autoencoder.fit(x_train_new, x_train_new, epochs=20)

In [ ]:
encoded_sample = autoencoder.encoder(x_train_new[:1,:,]).numpy()
decoded_sample = autoencoder.decoder(encoded_sample).numpy()
decoded_sample.shape

In [ ]:
import soundfile
mfcc_x = librosa.feature.inverse.mfcc_to_audio(decoded_sample.reshape(128, 20))
soundfile.write('example.wav', mfcc_x, 20000)

In [ ]:
ipd.Audio('example.wav')